In [1]:
import os
import sys

# Changes the current path to find the source files
current_dir = os.getcwd()
while current_dir != os.path.abspath("../src"):
    os.chdir("..")
    current_dir = os.getcwd()
sys.path.append(os.path.abspath("Efficient-Computing/Detection/Gold-YOLO"))

In [2]:
from layers import AMF_GD_YOLOv8
import torch
from utils import Folders
from data_processing import ImageData
from training_parameters import (
    class_names,
    class_indices,
    class_colors,
    transform_pixel,
    transform_spatial,
)
from training import TreeDataset, train_and_validate

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = (
    "garbage_collection_threshold:0.6,max_split_size_mb:512"
)
print(os.environ.get("PYTORCH_CUDA_ALLOC_CONF"))

garbage_collection_threshold:0.6,max_split_size_mb:512


In [4]:
model = AMF_GD_YOLOv8(3, 1, device=device, scale="n", class_names=class_names).to(
    device
)

# Paths to the data

full_image_path = "../data/images/full/2023_122000_484000_RGB_hrl.tif"
resolution = 0.08

image_data = ImageData(full_image_path)

annotations_folder_path = os.path.join(
    Folders.CROPPED_ANNOTS.value, image_data.base_name
)
rgb_folder_path = os.path.join(Folders.CROPPED_IMAGES.value, image_data.base_name)
chm_folder_path = os.path.join(
    Folders.CHM.value,
    f"{round(resolution*100)}cm",
    "unfiltered",
    "cropped",
    image_data.coord_name,
)

train_dataset = TreeDataset(
    annotations_folder_path=annotations_folder_path,
    rgb_folder_path=rgb_folder_path,
    chm_folder_path=chm_folder_path,
    labels_to_index=class_indices,
    labels_to_color=class_colors,
    transform_spatial=transform_spatial,
    transform_pixel=transform_pixel,
    # transform_spatial=None,
    # transform_pixel=None,
)

val_dataset = TreeDataset(
    annotations_folder_path=annotations_folder_path,
    rgb_folder_path=rgb_folder_path,
    chm_folder_path=chm_folder_path,
    labels_to_index=class_indices,
    labels_to_color=class_colors,
    transform_spatial=None,
    transform_pixel=None,
)

# Training parameters

lr = 1e-2
epochs = 100

batch_size = 2
num_workers = 2
accumulate = 10

final_model = train_and_validate(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    lr=lr,
    epochs=epochs,
    batch_size=batch_size,
    num_workers=num_workers,
    accumulate=accumulate,
    device=device,
)
state_dict = final_model.state_dict()
index = 1
while os.path.exists(f"../models/amf_gd_yolov8/trained_model_{index}.pt"):
    index += 1
torch.save(state_dict, f"../models/amf_gd_yolov8/trained_model_{index}.pt")

/home/alexandre/miniforge3/envs/tree-segment/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1716579264291/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


KeyboardInterrupt: 